In [ ]:
!wget 'https://www.gutenberg.org/files/11/old/alice30.txt'

--2021-01-12 03:50:20--  https://www.gutenberg.org/files/11/old/alice30.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 163218 (159K) [text/plain]
Saving to: ‘alice30.txt.2’

alice30.txt.2       100%[===================>] 159.39K  1019KB/s    in 0.2s    

2021-01-12 03:50:20 (1019 KB/s) - ‘alice30.txt.2’ saved [163218/163218]



In [ ]:
raw_text = open('/content/alice30new.txt', 'r', encoding='utf-8').read()

In [ ]:
raw_text

'\n                ALICE\'S ADVENTURES IN WONDERLAND\n\n                          Lewis Carroll\n\n               THE MILLENNIUM FULCRUM EDITION 3.0\n\n\n\n\n                            CHAPTER I\n\n                      Down the Rabbit-Hole\n\n\n  Alice was beginning to get very tired of sitting by her sister\non the bank, and of having nothing to do:  once or twice she had\npeeped into the book her sister was reading, but it had no\npictures or conversations in it, `and what is the use of a book,\'\nthought Alice `without pictures or conversation?\'\n\n  So she was considering in her own mind (as well as she could,\nfor the hot day made her feel very sleepy and stupid), whether\nthe pleasure of making a daisy-chain would be worth the trouble\nof getting up and picking the daisies, when suddenly a White\nRabbit with pink eyes ran close by her.\n\n  There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh d

In [ ]:
raw_text=raw_text.lower()

In [ ]:
import re

In [ ]:
raw_text = re.sub(r'[^\x00-\x7f]',r'', raw_text)
raw_text

'\n                alice\'s adventures in wonderland\n\n                          lewis carroll\n\n               the millennium fulcrum edition 3.0\n\n\n\n\n                            chapter i\n\n                      down the rabbit-hole\n\n\n  alice was beginning to get very tired of sitting by her sister\non the bank, and of having nothing to do:  once or twice she had\npeeped into the book her sister was reading, but it had no\npictures or conversations in it, `and what is the use of a book,\'\nthought alice `without pictures or conversation?\'\n\n  so she was considering in her own mind (as well as she could,\nfor the hot day made her feel very sleepy and stupid), whether\nthe pleasure of making a daisy-chain would be worth the trouble\nof getting up and picking the daisies, when suddenly a white\nrabbit with pink eyes ran close by her.\n\n  there was nothing so very remarkable in that; nor did alice\nthink it so very much out of the way to hear the rabbit say to\nitself, `oh d

In [ ]:
raw_text=raw_text.replace('\n',' ')



In [ ]:
my_new_string = re.sub('[^a-zA-Z0-9 \n\.]', '', raw_text)

In [ ]:
len(set(my_new_string))

30

In [ ]:
new1=set(my_new_string)

In [ ]:
list(new1)


['j',
 'e',
 'k',
 ' ',
 'f',
 'n',
 'p',
 'r',
 'q',
 'z',
 '3',
 'o',
 'c',
 'v',
 'h',
 'd',
 'w',
 '.',
 'm',
 'u',
 'g',
 'l',
 'y',
 'b',
 'x',
 'i',
 '0',
 's',
 'a',
 't']

In [ ]:
len(my_new_string)

141212

In [ ]:
char_indices = dict((c, i) for i, c in enumerate(new1))
indices_char = dict((i, c) for i, c in enumerate(new1))

In [ ]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(my_new_string) - maxlen, step):
    sentences.append(my_new_string[i: i + maxlen])
    next_chars.append(my_new_string[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 47058


In [ ]:
sentences

['                 alices adventures in wo',
 '              alices adventures in wonde',
 '           alices adventures in wonderla',
 '        alices adventures in wonderland ',
 '     alices adventures in wonderland    ',
 '  alices adventures in wonderland       ',
 'lices adventures in wonderland          ',
 'es adventures in wonderland             ',
 'adventures in wonderland                ',
 'entures in wonderland                   ',
 'ures in wonderland                      ',
 's in wonderland                         ',
 'n wonderland                            ',
 'onderland                            lew',
 'erland                            lewis ',
 'and                            lewis car',
 '                            lewis carrol',
 '                         lewis carroll  ',
 '                      lewis carroll     ',
 '                   lewis carroll        ',
 '                lewis carroll           ',
 '             lewis carroll              ',
 '        

In [ ]:
import numpy as np

In [ ]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(new1)), dtype=np.bool)
y = np.zeros((len(sentences), len(new1)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [ ]:
# split into train and test sets
train_size = int(len(x) * 0.9)
test_size = len(x) - train_size
xtrain, xtest = x[0:train_size,:], x[train_size:len(x),:]
ytrain, ytest = y[0:train_size,:], y[train_size:len(x),:]
x.shape, xtrain.shape, xtest.shape,y.shape,ytrain.shape,ytest.shape


((47058, 40, 30),
 (42352, 40, 30),
 (4706, 40, 30),
 (47058, 30),
 (42352, 30),
 (4706, 30))

In [ ]:
import numpy as np

In [ ]:
from keras.layers import Dense, LSTM
from keras.models import Sequential
model = Sequential()
num_features=30
model.add(LSTM(128, activation='tanh', input_shape=(40,num_features),return_sequences=False))
model.add(Dense(30, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               81408     
_________________________________________________________________
dense (Dense)                (None, 30)                3870      
Total params: 85,278
Trainable params: 85,278
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(lr=0.01), loss='categorical_crossentropy')
model.fit(xtrain,ytrain, verbose=1, epochs=10, validation_data=(xtest,ytest))

Epoch 1/10
1324/1324 [==============================] - 9s 6ms/step - loss: 1.1528 - val_loss: 1.6439
Epoch 2/10
1324/1324 [==============================] - 7s 5ms/step - loss: 1.1325 - val_loss: 1.6497
Epoch 3/10
1324/1324 [==============================] - 7s 5ms/step - loss: 1.1050 - val_loss: 1.6495
Epoch 4/10
1324/1324 [==============================] - 7s 5ms/step - loss: 1.1027 - val_loss: 1.6543
Epoch 5/10
1324/1324 [==============================] - 7s 5ms/step - loss: 1.1017 - val_loss: 1.6834
Epoch 6/10
1324/1324 [==============================] - 7s 5ms/step - loss: 1.0915 - val_loss: 1.6748
Epoch 7/10
1324/1324 [==============================] - 7s 5ms/step - loss: 1.0723 - val_loss: 1.6975
Epoch 8/10
1324/1324 [==============================] - 7s 5ms/step - loss: 1.0723 - val_loss: 1.7301
Epoch 9/10
1324/1324 [==============================] - 7s 5ms/step - loss: 1.0564 - val_loss: 1.6837
Epoch 10/10
1324/1324 [==============================] - 7s 6ms/step - loss: 1.057

In [ ]:
n1=np.reshape(xtest[0],(1,40,30))

In [ ]:
xtest[0]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
y_predict=model.predict(n1)

In [ ]:
np.argmax(y_predict)

29

In [ ]:
indices_char[11]

'o'

In [ ]:
m=''
for i in xtest[0]:
  a=np.argmax(i)
  b=indices_char[a]
  m=m+b
print(m)  


by the officers of the court.  as that i


In [ ]:
text='a new girl in town and persuades her to '
len(text)

40

In [ ]:
text='a new girl in town and persuades her to '
def sample(preds):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds) #exp of log (x), isn't this same as x??
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas)
for i in range(100):
  p = np.zeros((1, maxlen, len(new1)), dtype=np.bool)
#y = np.zeros((len(sentences), len(new1)), dtype=np.bool)
  for t, char in enumerate(text):
    p[0, t, char_indices[char]] = 1
  pred=model.predict(p)
  pred=sample(pred[0])
  pred=indices_char[pred]
  print(pred,end='')
  text=text+pred
  text=text[1:]


    

see and then your as he find the pool of the eird not anrace as dranced in the dire sried the march 

46